In [1]:
# Load a point cloud, display gt, detections, and optimized detections as bounding boxes
# Bouding box format: x y z l w h yaw

import argparse
import os
import sys
import numpy as np
import open3d as o3d


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
file_id = 100
print("file_id: ", file_id)
pcd_path = "/home/shashank/Documents/UniBonn/Sem2/MSR_P04/from_scratch/results/Argoverse/pcd_new/" + str(file_id) + ".pcd"
det_bbox_path = "/home/shashank/Documents/UniBonn/Sem2/MSR_P04/from_scratch/results/Argoverse/pcd/000/det/" + str(file_id) + ".txt" # yaw in radians
gt_bbox_path = "/home/shashank/Documents/UniBonn/Sem2/MSR_P04/from_scratch/results/Argoverse/pcd/000/gt000016/"+ str(file_id) + ".txt"  # yaw in degrees
opt_det_bbox_path = "/home/shashank/Documents/UniBonn/Sem2/MSR_P04/from_scratch/results/Argoverse/pcd_new_bouding_box_after_corrections/" + str(file_id) + ".txt"# yaw in radians



file_id:  100


In [3]:
gt_bbox = np.loadtxt(gt_bbox_path)
det_bbox = np.loadtxt(det_bbox_path)
opt_det_bbox = np.loadtxt(opt_det_bbox_path)
# Rotation matrix from yaw
R_gt_bbox = np.array([[np.cos(gt_bbox[6]), -np.sin(gt_bbox[6]), 0],
                        [np.sin(gt_bbox[6]), np.cos(gt_bbox[6]), 0],
                        [0, 0, 1]])
R_det_bbox = np.array([[np.cos(np.deg2rad(det_bbox[6])), -np.sin(np.deg2rad(det_bbox[6])), 0],
                        [np.sin(np.deg2rad(det_bbox[6])), np.cos(np.deg2rad(det_bbox[6])), 0],
                        [0, 0, 1]])
R_opt_det_bbox = np.array([[np.cos(opt_det_bbox[6]), -np.sin(opt_det_bbox[6]), 0],
                        [np.sin(opt_det_bbox[6]), np.cos(opt_det_bbox[6]), 0],
                        [0, 0, 1]])
# Complete transformation matrix
T_gt_bbox = np.eye(4)
T_det_bbox = np.eye(4)
T_opt_det_bbox = np.eye(4)
T_gt_bbox[:3, :3] = R_gt_bbox
T_det_bbox[:3, :3] = R_det_bbox
T_opt_det_bbox[:3, :3] = R_opt_det_bbox
T_gt_bbox[:3, 3] = gt_bbox[:3]
T_det_bbox[:3, 3] = det_bbox[:3]
T_opt_det_bbox[:3, 3] = opt_det_bbox[:3]




In [4]:

# Load point cloud
pcd_folder = "/home/shashank/data/datasets/av2/train-000/train/074d2237-ed1b-34d7-a2fc-68edbce50bb2/sensors/lidar_pcd"
# Make pcd file list from folder
pcd_list = []
for file in os.listdir(pcd_folder):
    if file.endswith(".pcd"):
        pcd_list.append(os.path.join(pcd_folder, file))
# pcd_list.sort()
# for file in pcd_list:
#     print(os.path.basename(file))
pcd_full_path = pcd_list[file_id]
pcd_full = o3d.io.read_point_cloud(pcd_full_path)
# o3d.visualization.draw_geometries([pcd_full])

In [5]:
# Create gt bbox - open3d object
gt_bbox_obj = o3d.geometry.OrientedBoundingBox()
gt_bbox_obj.color = (0, 1, 0)
gt_bbox_obj.center = gt_bbox[:3]
gt_bbox_obj.R = R_gt_bbox
gt_bbox_obj.extent = gt_bbox[3:6]

# Create det bbox - open3d object
det_bbox_obj = o3d.geometry.OrientedBoundingBox()
det_bbox_obj.color = (1, 0, 0)
det_bbox_obj.center = det_bbox[:3]
det_bbox_obj.R = R_det_bbox
det_bbox_obj.extent = det_bbox[3:6]

# Create opt det bbox - open3d object
opt_det_bbox_obj = o3d.geometry.OrientedBoundingBox()
opt_det_bbox_obj.color = (0, 0, 1)
opt_det_bbox_obj.center = opt_det_bbox[:3]
opt_det_bbox_obj.R = R_opt_det_bbox
opt_det_bbox_obj.extent = opt_det_bbox[3:6]

# Visualize
o3d.visualization.draw_geometries([pcd_full, gt_bbox_obj, det_bbox_obj, opt_det_bbox_obj])



In [6]:
# Visualize in det frame

pcd_det_frame_path = "/home/shashank/Documents/UniBonn/Sem2/MSR_P04/from_scratch/results/Argoverse/pcd_new/" + str(file_id) + ".pcd"
pcd_det_frame  = o3d.io.read_point_cloud(pcd_det_frame_path)

T_gt_wrt_det = np.linalg.inv(T_det_bbox) @ T_gt_bbox
T_opt_det_wrt_det = np.linalg.inv(T_det_bbox) @ T_opt_det_bbox

gt_bbox_det_frame = o3d.geometry.OrientedBoundingBox()
gt_bbox_det_frame.color = (0, 1, 0)
gt_bbox_det_frame.center = T_gt_wrt_det[:3, 3]
gt_bbox_det_frame.R = T_gt_wrt_det[:3, :3]
gt_bbox_det_frame.extent = gt_bbox[3:6]

opt_det_bbox_det_frame = o3d.geometry.OrientedBoundingBox()
opt_det_bbox_det_frame.color = (0, 0, 1)
opt_det_bbox_det_frame.center = T_opt_det_wrt_det[:3, 3]
opt_det_bbox_det_frame.R = T_opt_det_wrt_det[:3, :3]
opt_det_bbox_det_frame.extent = opt_det_bbox[3:6]

det_bbox_det_frame = o3d.geometry.OrientedBoundingBox()
det_bbox_det_frame.color = (1, 0, 0)
det_bbox_det_frame.center = np.zeros(3)
det_bbox_det_frame.R = np.eye(3)
det_bbox_det_frame.extent = det_bbox[3:6]

# o3d.visualization.draw_geometries([pcd_det_frame, gt_bbox_det_frame, det_bbox_det_frame, opt_det_bbox_det_frame])


In [7]:
# Visualizing with mesh 
dict_path = "/home/shashank/Documents/UniBonn/Sem2/MSR_P04/from_scratch/results/Argoverse/pcd_new_bouding_box_corrections/" + str(file_id) + ".npy"
dict = np.load(dict_path, allow_pickle=True).item()
mesh_pcd = dict['mesh_object_frame_pcd']
# Numpy to open3d
mesh_pcd_o3d = o3d.geometry.PointCloud()
mesh_pcd_o3d.points = o3d.utility.Vector3dVector(mesh_pcd)
mesh_pcd_o3d.paint_uniform_color([1, 0.0, 1])
# Visualize
o3d.visualization.draw_geometries([pcd_det_frame, gt_bbox_det_frame, det_bbox_det_frame, opt_det_bbox_det_frame, mesh_pcd_o3d])